In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

In [4]:
df=pd.read_csv('sphist.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [5]:
df["Date"]=pd.to_datetime(df["Date"])
df=df.sort_values(by="Date")
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


In [23]:
for i, row in df.iterrows():
    row['day_5']=0
    row['day_5_std']=0
    row['day_30']=0
    row['day_30_std']=0
    if (i-5)>=0:          
        five_day_data=[df['Close'].iloc[j] for j in range (i-5,i)]
        row['day_5']=np.array(five_day_data).mean()
        row['day_5_std']=np.array(five_day_data).std()
        df.loc[i,'day_5']=row['day_5']
        df.loc[i,'day_5_std']=row['day_5_std']
    if (i-30)>=0:          
        thirty_day_data=[df['Close'].iloc[j] for j in range (i-30,i)]
        row['day_30']=np.array(thirty_day_data).mean()
        row['day_30_std']=np.array(thirty_day_data).std()
        df.loc[i,'day_30']=row['day_30']
        df.loc[i,'day_30_std']=row['day_30_std']
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,day_5,day_5_std,day_30,day_30_std
16339,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,2080.771973,17.715146,2080.237329,19.345695
16338,1951-01-04,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,2080.456006,17.530725,2078.931331,19.844521
16337,1951-01-05,20.870001,20.870001,20.870001,20.870001,3390000.0,20.870001,2088.306006,8.348225,2077.908659,22.001966
16336,1951-01-08,21.000000,21.000000,21.000000,21.000000,2780000.0,21.000000,2090.231982,7.116786,2076.283993,23.567559
16335,1951-01-09,21.120001,21.120001,21.120001,21.120001,3800000.0,21.120001,2087.024023,3.502675,2073.984998,24.239795


In [30]:
df.iloc[0]

Date          1951-01-03 00:00:00
Open                        20.69
High                        20.69
Low                         20.69
Close                       20.69
Volume                   3.37e+06
Adj Close                   20.69
day_5                     2080.77
day_5_std                 17.7151
day_30                    2080.24
day_30_std                19.3457
Name: 16339, dtype: object

In [24]:
df=df[df["Date"] >= datetime(year=1951, month=1, day=3)].dropna(axis=0)

In [25]:
train=df[df["Date"] < datetime(year=2013, month=1, day=1)]
test=df[df["Date"] >= datetime(year=2013, month=1, day=1)]

In [27]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train[['day_5','day_5_std','day_30']], train['Close'])
predictions=lr.predict(test[['day_5','day_5_std', 'day_30']])
mae=sum(abs(predictions-test['Close']))/len(predictions)
print(mae)

1192.84244354
